#                            **Bernoulli Naive Bayes Classifier from scratch ** 
>>>> ***Spam/Ham classification***






This project consists of classifying whether a given message is a spam or a ham.
This is possible with a function given by sklearn but the goal is to achieve the same result with a function made from scratch in order to understand the mathematical approach.

To solve this problem, we first construct a dictionary and generate a feature vector that corresponds to the given message. 
The dictionary is constructed by listing all the words contained in a collection of messages. 
Then, we identify the words contained in a given message by generating the feature vector.


**$\begin{aligned} P \left( x _ { 1 } , \cdots , x _ { 500000 } | y \right) & = P \left( x _ { 1 } | y \right) P \left( x _ { 2 } | y , x _ { 1 } \right) P \left( x _ { 3 } | y , x _ { 1 } , x _ { 2 } \right) \cdots \\ & = P \left( x _ { 1 } | y \right) P \left( x _ { 2 } | y \right) P \left( x _ { 3 } | y \right) \cdots P \left( x _ { 50000 } | y \right) \\ & = \prod _ { n = 1 } ^ { N } P \left( x _ { n } | y \right) \end{aligned}$**

The above expression says that once we know that a message is Spam (or not), the presence of some words in this message does not help to know whether some other words are also present in that message.
This (naive) assumption is obviously false. But, despite this incongruence, the Naive Bayes algorithm works well for classifying texts. On the other hand, without this assumption the problem to be solved would be much harder.


**Mathematical formulas : **




* Joint likelyhood : 





$\mathcal { L } \left( \phi _ { y } , \phi _ { j | y = 1 } , \phi _ { j | y = 0 } \right) = \prod _ { i = 1 } ^ { I } P \left( x ^ { ( i ) } , y ^ { ( i ) } \right)$


---



* Maximum Likelihood Estimation (MLE) :



$\phi _ { y } = \frac { 1 } { I } \sum _ { i = 1 } ^ { I } \mathbb { I } \left\{ y ^ { ( i ) } = 1 \right\}$

$\phi _ { n | y = 1 } = \frac { \sum _ { i = 1 } ^ { I } \mathbb { I } \left\{ x _ { n } ^ { ( i ) } = 1 , y ^ { ( i ) } = 1 \right\} } { \sum _ { i = 1 } ^ { I } \mathbb { I } \left\{ y ^ { ( i ) } = 1 \right\} }$

$\phi _ { n | y = 0 } = \frac { \sum _ { i = 1 } ^ { I } \mathbb { I } \left\{ x _ { n } ^ { ( i ) } = 1 , y ^ { ( i ) } = 0 \right\} } { \sum _ { i = 1 } ^ { I } \mathbb { I } \left\{ y ^ { ( i ) } = 0 \right\} }$




---


* Predict criterion : 

$\underset { y } { \arg \max } P ( y | x ) = \underset { y } { \arg \max } \frac { P ( x | y ) P ( y ) } { P ( x ) } = \underset { y } { \arg \max } P ( x | y ) P ( y )$



---

**Bayes rule : **

$P ( A | B ) = \frac { P ( B | A ) P ( A ) } { P ( B ) }$


**Importing the needed libraries : **

In [0]:
import os,sys
import numpy as np
from collections import Counter
import math
from sklearn.metrics import confusion_matrix

In [0]:
def read_data(filename):
    """
    Read data 'line by line', using generators.
    Generators make it easier to process BIG text files.
    """
    with open(filename, 'r',encoding="utf8", errors='ignore') as input:
        for line in input:
            yield line
            

In [0]:
def ham_split(data):
  for i in data:
    i=i.split("\t")
    if i[0]=='ham':
      yield i[1]

def spam_split(data):
  for i in data:
    i=i.split("\t")
    if i[0]=='spam':
      yield i[1]

def get_target(data) : 
  liste=[]
  for i in data: 
    i=i.split("\t")
    if i[0]=='ham' :
      liste.append(0)
    else :
      liste.append(1)
  return liste

## **Reading the data**

In [0]:
data_file="messages.txt"

#Accessing the data file
data = read_data(data_file)
data_list=list(data)

#Splitting the data into training and test
#Ratio used : 70%-30%
size = len(data_list)
training_size=int(0.7*size)
test_size=int(0.3*size)

training_data=data_list[:training_size]
test_data=data_list[training_size:training_size+test_size]


#Extracting the messages into list of hams and list of spams

ham_training=list(ham_split(training_data))
ham_test=list(ham_split(test_data))
spam_training=list(spam_split(training_data))
spam_test=list(spam_split(test_data))

#Associated target lists : 
# Ham : 0
# Spam : 1

training_target_matrix=get_target(training_data)
test_target_matrix=get_target(test_data)

## **Creating a dictionary**

In [0]:
#Make a dictionary based on message's words, return the most common 3000 words
def make_Dictionary(data):
    all_words = []
    for line in data:
      words = line.split()
      all_words += words
    for w in all_words: 
        if w.isalpha() == False:
          for i in range(all_words.count(w)):
            all_words.remove(w)
        elif len(w) == 1:
          for i in range(all_words.count(w)):
            all_words.remove(w)
    all_words=[x.lower() for x in all_words]
    dictionary = Counter(all_words)
    dictionary = dictionary.most_common(3000)
    return dictionary

In [0]:
#Creating dictionary based on the training data
clean_training_data=ham_training+spam_training
clean_test_data=ham_test+spam_test

training_dictionary=make_Dictionary(clean_training_data)

## **Modeling the data **

In [0]:
#Create a matrix with the occurences of each features for every messages, based on the dictionary
def extract_features(data,dictionary):
    #files = [os.path.join(mail_dir, fi) for fi in os.listdir(mail_dir)]
    features_matrix = np.zeros((len(data), 3000))
    lineID = 0
    for line in data:
      words = line.split()
      for word in words:
        wordID = 0
        for i, d in enumerate(dictionary):
          if d[0] == word:
            wordID = i
            features_matrix[lineID, wordID] = 1 #Specific to Bernoulli
      lineID = lineID + 1
    return features_matrix

In [10]:
#Extracting features from the training data and the test data
train_matrix = extract_features(clean_training_data,training_dictionary)
test_matrix = extract_features(clean_test_data,training_dictionary)


KeyboardInterrupt: ignored

In [0]:
def fit(matrix_input, matrix_target) :
  true_probability = matrix_target.count(1)/len(matrix_target)
  false_probability = 1 - true_probability
  matrix_prob_positive = []
  matrix_prob_negative = []
  ham_count=0
  spam_count=0
  columns = [l for l in zip(*matrix_input)]
  for i in range(len(columns)): #i->columns
    for j in range(len(columns[i])): #j->rows
      if matrix_target[j]==1:
        spam_count=spam_count+columns[i][j]
      else :
        ham_count=ham_count+columns[i][j]
    matrix_prob_positive.append((spam_count*true_probability)/matrix_target.count(1))
    matrix_prob_negative.append((ham_count*false_probability)/matrix_target.count(0))
    ham_count=0
    spam_count=0
  matrix_prob=[]
  matrix_prob.append(matrix_prob_positive)
  matrix_prob.append(matrix_prob_negative)
  return matrix_prob

In [0]:
def predict(matrix_prob,matrix_test) :
  result=[]
  for i in range(len(matrix_test)):
    pos_prob=1
    neg_prob=1
    for j in range(len(matrix_test[i])):
      if matrix_test[i][j]==1:
        pos_prob=pos_prob*(matrix_test[i][j]*matrix_prob[0][j])
        neg_prob=neg_prob*(matrix_test[i][j]*matrix_prob[1][j])
    decision = np.argmax([neg_prob,pos_prob])
    if decision == 0 :
      result.append(0)
    elif decision == 1:
      result.append(1)
  return result

## **Comparing results from the sklearn function and the created function**

### sklearn function : 

In [0]:
from sklearn.naive_bayes import BernoulliNB

model = BernoulliNB()
model.fit(train_matrix, training_target_matrix)
sklearn_result = model.predict(test_matrix)
sklearn_cm=confusion_matrix(test_target_matrix, sklearn_result)

### **created function :**

In [0]:
result=predict(fit(train_matrix, training_target_matrix),test_matrix)
cm=confusion_matrix(test_target_matrix, result)

### Comparing : 

In [0]:
sklearn_score =(sklearn_cm[0,0]+sklearn_cm[1,1])/(sklearn_cm[0,0]+sklearn_cm[0,1]+sklearn_cm[1,0]+sklearn_cm[1,1])
score = (cm[0,0]+cm[1,1])/(cm[0,0]+cm[0,1]+cm[1,0]+cm[1,1])

print("sklearn score : ", round(sklearn_score*100, 2),"%")
print("created function score : ", round(score*100, 2),"%")
print("Accuracy : ", round( ((sklearn_score/score)*100) ,2),"%")